In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
import math
import hanja
import re
import string
import operator
import random
import matplotlib.pyplot as plt
import itertools
import cnouns as cn
import check_utils as cu
import deep_utils as du
from sklearn.metrics import adjusted_rand_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from datetime import datetime
from sklearn.decomposition import PCA
from gensim import models
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
import gensim.models.doc2vec
from collections import OrderedDict
from gensim.models.doc2vec import LabeledSentence

import multiprocessing
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
import cPickle as pickle
from spherecluster import SphericalKMeans
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

from random import shuffle

from collections import namedtuple

Articles = namedtuple('Articles', 'words tags split')

# Load Data

In [2]:
stop = 0
# test = 1
test = 2

In [3]:
train_df = pd.read_pickle("../datastore/deep_df.p")

models_by_name = OrderedDict()
models_by_name['Doc2Vec(dm/c,d100,n5,w5,mc2,t8)'] = Doc2Vec.load("../datastore/d2v-dmc_%d.p" % test)
models_by_name['Doc2Vec(dbow,d100,n5,mc2,t8)'] = Doc2Vec.load("../datastore/d2v-dbow_%d.p" % test)
models_by_name['Doc2Vec(dm/m,d100,n5,w10,mc2,t8)'] = Doc2Vec.load("../datastore/d2v-dmm_%d.p" % test)

In [4]:
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([models_by_name['Doc2Vec(dbow,d100,n5,mc2,t8)'], models_by_name['Doc2Vec(dm/m,d100,n5,w10,mc2,t8)']])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([models_by_name['Doc2Vec(dbow,d100,n5,mc2,t8)'], models_by_name['Doc2Vec(dm/c,d100,n5,w5,mc2,t8)']])

# Select Model

In [5]:
# dm = models_by_name['dbow+dmm']
# dm = models_by_name['dbow+dmc']
dm = models_by_name['Doc2Vec(dm/c,d100,n5,w5,mc2,t8)']

In [6]:
doc_arr = dm.docvecs
inp = np.array(doc_arr)

# Clustering

In [7]:
n_cluster = 200

model = KMeans
# model = SphericalKMeans

In [8]:
d_km = model(n_clusters=n_cluster, n_jobs=-1)
d_km.fit(inp)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=200, n_init=10,
    n_jobs=-1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [9]:
clusters = d_km.labels_.tolist()
train_df['cluster'] = clusters

# Scoring

In [10]:
print "inertia : ", d_km.inertia_
print "silhouette score : ", silhouette_score(inp, d_km.labels_)

inertia :  63355.3607526
silhouette score :  0.0942619959317


In [11]:
sorted_cluster = cu.sort_count(train_df, range(n_cluster))
sorted_cluster

[(8, 169),
 (163, 93),
 (194, 61),
 (191, 48),
 (78, 45),
 (110, 44),
 (14, 42),
 (173, 40),
 (6, 39),
 (158, 39),
 (73, 37),
 (93, 37),
 (145, 37),
 (28, 36),
 (61, 35),
 (105, 35),
 (23, 34),
 (25, 34),
 (156, 34),
 (31, 33),
 (55, 29),
 (19, 27),
 (15, 26),
 (27, 26),
 (132, 26),
 (149, 26),
 (94, 25),
 (101, 25),
 (166, 25),
 (102, 24),
 (128, 24),
 (68, 23),
 (7, 22),
 (10, 22),
 (69, 22),
 (32, 21),
 (4, 20),
 (181, 20),
 (16, 19),
 (111, 19),
 (120, 19),
 (21, 18),
 (46, 18),
 (12, 17),
 (53, 17),
 (74, 17),
 (87, 17),
 (135, 17),
 (9, 16),
 (37, 16),
 (114, 16),
 (142, 16),
 (64, 15),
 (169, 15),
 (176, 15),
 (17, 14),
 (22, 14),
 (2, 13),
 (54, 13),
 (83, 13),
 (127, 13),
 (165, 13),
 (30, 12),
 (44, 12),
 (118, 12),
 (130, 12),
 (0, 11),
 (29, 11),
 (98, 11),
 (18, 10),
 (26, 10),
 (43, 10),
 (141, 10),
 (159, 10),
 (45, 9),
 (75, 9),
 (148, 9),
 (11, 8),
 (35, 8),
 (48, 8),
 (56, 8),
 (62, 8),
 (85, 8),
 (100, 8),
 (108, 8),
 (3, 7),
 (20, 7),
 (57, 7),
 (58, 7),
 (72, 7),
 

In [12]:
cu.test_print(32, d_km, inp, train_df)

1781 [날씨] 아침 안개 ‘주의’…낮에는 30도 안팎 늦더위
size  21
79                  추석 귀성길, 14일 오전에 가장 막힌다
80               지하철·버스, 16~17일 새벽 2시까지 운행
83                   오늘 백로… 늦더위 가고 미세먼지 온다
191                 오늘까지만 늦더위… 내일부터 초가을 날씨
324                               추석은 피하세요
562                          남부·동해안 스치는 태풍
755             귀성 14일 오전―귀경 15일 오후 가장 막힌다
949                 올 추석 보름달 둥실… 연휴 비소식 없어
1300        태풍 ‘남테운’ 영향… 9월 첫째주말 남해안에 많은 비
1399           8월, 폭염 16.7일 역대 최장… 강수량은 최저
1573    16~17일 추석연휴에 시내버스·지하철 새벽 2시까지 연장운행
1611      [날씨] 농작물에 이슬 맺힌다는 ‘백로’…전국 따사로운 날
1742     추석 연휴 16~17일 서울 지하철·버스 새벽 2시까지 운행
1746                  추석연휴 고속도로 가장 혼잡한 날은?
1781        [날씨] 아침 안개 ‘주의’…낮에는 30도 안팎 늦더위
1959         “한가위 보름달 볼 수 있다…연휴에 비 예보도 없어”
1969                      [날씨] 오늘도 좀 덥겠습니다
2086               [날씨] 흐린 가을 하늘에서 비가 내립니다
2197          [날씨] 충청 이남에 강한 비…낮 기온 23∼27도
2319            2~3일 남부지방과 동해안에 50~100㎜ 폭우
2347             8월 보름 넘게 폭염 역대 1위…강수량은 꼴찌
Name: title, dtype: object


In [13]:
print dm.docvecs.similarity(d1=2183, d2=2165)
print dm.docvecs.similarity(d1=2267, d2=2328)

0.65338858722
0.756016786464


In [14]:
cu.test_similar(1, dm.docvecs, train_df, threadsold=0.5, is_last = False)

most_similar - 2343
target -  [카드뉴스] 그의 수퍼카 부가티는 개미들의 눈물로 달렸을까요?

통일나눔펀드에 동참하세요
통일나눔펀드에 동참하세요
통일나눔펀드에 동참하세요
[카드뉴스] 배달 앱에 꺾인 프랑스의 '요리부심'
[오늘의 운세/9월7일]
9월 7일 김영훈의 생각줍기
[카드뉴스] 퇴비 수거 현장에 부시장님까지 출동한 사연
통일나눔펀드에 동참하세요
[오늘의 운세/9월6일]
[오늘의 운세/9월 4일]
[오늘의 경기] 2016년 9월 7일
[오늘의 경기] 2016년 9월 2일
[오늘의 운세/9월2일]
[오늘의 경기/9월3일]프로야구 外
9월 6일 김영훈의 생각줍기
[카드뉴스] "아가야, 네가 꼭 세상을 살아보게 해줄게."
[오늘의 운세/9월5일]
한승수 前 총리 중어판 연설문집 출간
[오늘의 경기] 2016년 9월 5일
[입에 착착 붙는 일본어] 대역을 완수하다
광화문서 열린 핼러윈 퍼레이드
[카드뉴스] 한 소년이 15세 생일에 받은 축하 편지
[이번주 개봉영화 딱 10자평] '머니몬스터' 외
[인사]환경부
[청년드림/알립니다]청년드림 도시락토크 2.0… 주니어사원과 점심을
[리빙포인트] 전기밥솥으로 맥반석 달걀 만들기
9월 6일 한겨레 그림판
[오늘의 운세/9월 3일]
9월 5일 궂긴소식
[오늘의 경기] 2016년 9월 3일
9월 7일 한겨레 그림판
9월 2일 궂긴 소식
[카드뉴스] 천년 고도 경주를 사극 세트장으로?
통일나눔펀드에 동참하세요
홍승연, 영월 국제여자서키트 우승
[리빙포인트] 벽지에 묻은 얼룩 지우려면
[Campus&]대학기업가센터… 캠퍼스 창업 생태계 구축의 구심점
김혜나 수림문학상
[카드뉴스] '교실 밖' 아이들을 알고 싶었던 선생님의 질문
[지표로 보는 경제]9월 6일
[카드뉴스] 애플이 16조원 '세금폭탄'을 맞은 까닭.
[미술 소식] 23일까지 '최덕휴, 도시와 바다' 展
9월 2일 한겨레 그림판
[리빙포인트] 새 프라이팬 오래 쓰려면
[내일의 경기/9월4일]프로야구 外
[스포츠 브리핑] 강정호, 재활 경기서 홈

# Stop

In [15]:
if(stop==1):
    asdasd

# Similarity Clustering

In [16]:
train_df = pd.read_pickle("../datastore/deep_result_df.p")
centers = pickle.load(open("../datastore/deep_centers.p", "rb"))

In [17]:
du.calc_similarity(train_df, dm.docvecs, centers)

In [18]:
train_df[train_df.similarity<1.0]

,_id,category,content,description,provider,providerNewsID,publishedAt,title,target_str,cluster,rank,similarity
0,57e2c849149c2181df5b92aa,nativeAD > CJ,CJ그룹이 7일부터 하반기 신입사원 1700명을 공개 채용한다. 작년보다 18% 늘...,,chosun,2016090701420,2016-09-07 11:31:00,"CJ그룹, 하반기 신입사원 1700명 공개 채용",그룹NNG 하반기NNG 신입NNG 사원NNG 공개NNG 채용NNG 그룹NNG 하반기...,944,0.0,0.874569
1,57e2c849149c2181df5b92ac,사회 > 카드뉴스,흙수저 출신 20대 억만장자의 '엉터리' 조언 기사보기 : http://goo.g...,,chosun,2016090701310,2016-09-07 10:59:00,[카드뉴스] 그의 수퍼카 부가티는 개미들의 눈물로 달렸을까요?,수퍼NNP 카NNP 부가티NNP 개미NNG 눈물NNG 흙NNG 수저NNG 출신NNG...,2,0.0,0.641958
2,57e2c849149c2181df5b92ad,사회 > 카드뉴스,호주 역사상 처음!\n\n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조...,,chosun,2016090701283,2016-09-07 10:49:00,[카드뉴스] 열한살 소년이 지폐를 바꿨습니다,소년NNG 지폐NNG 호주NNP 역사NNG 호주NNP역사NNG 소년NNG이JKS지폐...,2,1.0,0.697542
3,57e2c849149c2181df5b92ae,사회 > 카드뉴스,한국프로야구 승부조작 \n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조...,,chosun,2016090701173,2016-09-07 10:23:00,[카드뉴스] '원조 승부조작' 前 야구선수의 참회,원조NNG 승부조작NNP 야구NNG 선수NNG 참회NNG 한국NNP 프로야구NNP ...,2,0.0,0.671793
4,57e2c849149c2181df5b92b0,사설ㆍ칼럼 > 내부칼럼 > 사설,추미애 더불어민주당 대표가 5일 취임 이후 첫 국회 교섭단체 연설에서 청와대에 '비...,,chosun,2016090603334,2016-09-07 03:23:00,[사설] 경제 앞길 막은 야당이 경제 失政 비판할 자격 있나,경제NNG 앞길NNG 야당NNG 경제NNG 실정NNG 비판NNG 자격NNG 추미NN...,280,0.0,0.675553
5,57e2c849149c2181df5b92b1,사설ㆍ칼럼 > 내부칼럼 > 사설,금융 공기업에 파견 근무 중인 김모 부장검사가 자신의 스폰서 역할을 해온 고교 동창...,,chosun,2016090603336,2016-09-07 03:19:00,"[사설] 檢察선 또 스폰서 검사, 法院선 또 돈 받고 재판",검찰NNG 스폰서NNG 검사NNG 법원NNG 돈NNG 재판NNG 금융NNG 공기업N...,90,0.0,0.821865
6,57e2c84a149c2181df5b92b2,사설ㆍ칼럼 > 내부칼럼 > 사설,의료기관에서 C형 간염 집단감염 사태가 잇따르자 정부가 6일 C형 간염을 전수(全數...,,chosun,2016090603338,2016-09-07 03:18:00,"[사설] C형 간염, 의료기 再사용 병·의원 명단 공개해야",간염NNG 의료기NNG 사용NNG 병NNG 의원NNG 명단NNG 공개NNG 의료NN...,66,0.0,0.715886
8,57e2c84a149c2181df5b92b5,사설ㆍ칼럼 > 내부칼럼 > 만물상,부산 초읍동 어린이대공원 뒤 백양산을 오르다 보면 높이 27m 거대한 석벽을 마주한...,,chosun,2016090603367,2016-09-07 03:16:00,[만물상] 음수사원(飮水思源),수사원NNG 글자NNG 눈NNG 물NNG 때NNG 수원지NNG 사람NNG 노고NNG...,63,0.0,0.660469
9,57e2c84a149c2181df5b92b6,사설ㆍ칼럼 > 외부칼럼 > 정민의 세설신어,유관현(柳觀鉉·1692~1764)은 1759년 필선(弼善)의 직책으로 사도세자를 3...,,chosun,2016090603374,2016-09-07 03:15:00,[정민의 世說新語] [382] 난자이사 (難者二事),난자NNG 이사NNG 대목NNG 가난NNG 부자NNG 의리NNG 돈NNG 공NNG ...,2,0.0,0.627997
11,57e2c84b149c2181df5b92b9,사설ㆍ칼럼 > 내부칼럼 > 특파원 칼럼,지난달 훈련 중이던 스위스 공군 전투기가 알프스에서 실종됐다. 산맥 어딘가에 추락했...,,chosun,2016090603343,2016-09-07 03:13:00,[특파원 리포트] 나치를 떨게 한 스위스의 정신 방위,나치NNG 스위스NNP 정신NNG 방위NNG 지난달NNG 훈련NNG 스위스NNP 공...,11,0.0,1.000000


# Similarity Scoring

In [19]:
score = du.similarity_iner_score(centers, train_df, dm.docvecs)

In [20]:
size_1 = score[score.cnt==1]
countby = score[score.cnt>10]
print "total:", len(score), ", size_1:",len(size_1), ", countby:", len(countby)
ss = countby.sum(axis=0)
print "distance:", ss['distance'] * 100
print "variance:", ss['variance']
print "similarity:", (ss['similarity'] * 100)/len(countby)
print "cohesion:", ss['cohesion']

total: 981 , size_1: 780 , countby: 25
distance: 21799.2317173
variance: 4.63020754543
similarity: 82.3610049993
cohesion: 114.300926309


In [21]:
score.sort_values('similarity', ascending=True)[:10]

,cluster,cnt,distance,variance,similarity,cohesion
1,2,265,68.801445,0.259628,0.674026,2.596123
246,1246,16,3.675626,0.229727,0.721876,3.142327
2,280,65,14.915201,0.229465,0.745912,3.250662
6,63,91,19.643313,0.215861,0.774204,3.586593
4,66,29,5.921117,0.204176,0.792256,3.880253
21,378,64,13.209853,0.206404,0.792945,3.841716
69,1159,16,3.156414,0.197276,0.797317,4.041634
67,245,55,10.690684,0.194376,0.812098,4.177972
35,674,14,2.569750,0.183554,0.820075,4.467771
18,688,101,19.042969,0.188544,0.826023,4.381057


In [22]:
score.sort_values('cnt', ascending=False)[:10]

,cluster,cnt,distance,variance,similarity,cohesion
1,2,265,68.801445,0.259628,0.674026,2.596123
18,688,101,19.042969,0.188544,0.826023,4.381057
6,63,91,19.643313,0.215861,0.774204,3.586593
2,280,65,14.915201,0.229465,0.745912,3.250662
21,378,64,13.209853,0.206404,0.792945,3.841716
67,245,55,10.690684,0.194376,0.812098,4.177972
41,180,49,7.029059,0.143450,0.887306,6.185463
3,90,34,5.669729,0.166757,0.859761,5.155779
68,128,33,5.327236,0.161431,0.870071,5.389726
4,66,29,5.921117,0.204176,0.792256,3.880253


In [23]:
countby.sort_values('variance', ascending=True)[:10]

,cluster,cnt,distance,variance,similarity,cohesion
41,180,49,7.029059,0.143450,0.887306,6.185463
65,923,11,1.644045,0.149459,0.888858,5.947185
73,468,19,2.938182,0.154641,0.875896,5.664054
284,786,11,1.754988,0.159544,0.873182,5.472974
54,361,14,2.245131,0.160366,0.853959,5.325049
68,128,33,5.327236,0.161431,0.870071,5.389726
3,90,34,5.669729,0.166757,0.859761,5.155779
198,580,24,4.035070,0.168128,0.860332,5.117126
47,80,22,3.763839,0.171084,0.842784,4.926154
294,588,20,3.512549,0.175627,0.841200,4.789682


In [24]:
countby.sort_values('similarity', ascending=False)[:10]

,cluster,cnt,distance,variance,similarity,cohesion
65,923,11,1.644045,0.149459,0.888858,5.947185
41,180,49,7.029059,0.143450,0.887306,6.185463
73,468,19,2.938182,0.154641,0.875896,5.664054
284,786,11,1.754988,0.159544,0.873182,5.472974
68,128,33,5.327236,0.161431,0.870071,5.389726
198,580,24,4.035070,0.168128,0.860332,5.117126
3,90,34,5.669729,0.166757,0.859761,5.155779
54,361,14,2.245131,0.160366,0.853959,5.325049
47,80,22,3.763839,0.171084,0.842784,4.926154
294,588,20,3.512549,0.175627,0.841200,4.789682


In [25]:
countby.sort_values('cohesion', ascending=False)[:10]

,cluster,cnt,distance,variance,similarity,cohesion
41,180,49,7.029059,0.143450,0.887306,6.185463
65,923,11,1.644045,0.149459,0.888858,5.947185
73,468,19,2.938182,0.154641,0.875896,5.664054
284,786,11,1.754988,0.159544,0.873182,5.472974
68,128,33,5.327236,0.161431,0.870071,5.389726
54,361,14,2.245131,0.160366,0.853959,5.325049
3,90,34,5.669729,0.166757,0.859761,5.155779
198,580,24,4.035070,0.168128,0.860332,5.117126
47,80,22,3.763839,0.171084,0.842784,4.926154
294,588,20,3.512549,0.175627,0.841200,4.789682


# Get Topics

In [26]:
topics = pickle.load(open("../datastore/deep_topics.p", "rb"))

In [27]:
for idx, row in countby.sort_values('cohesion', ascending=False)[:10].iterrows():
    print du.test_print(row.cluster, train_df, dm.docvecs, centers, topics, countby)
    print "\n------------------------------------------------------------\n"

    cluster  cnt  distance  variance  similarity  cohesion
41      180   49  7.029059   0.14345    0.887306  6.185463

Most similar - [부고]송두만 별세·김대영 민녕 영애 영숙 영자 영명 모친상·최융호 손명원 허광수 정몽준 장모상·정다미 시모상

 발인NNG 별세NNG 서울NNP
 발인NNG 병원NNG 부친상NNG
 발인NNG 모친상NNG 오전NNG

1309 0.995528876781 [부고]송두만 별세·김대영 민녕 영애 영숙 영자 영명 모친상·최융호 손명원 허광수 정몽준 장모상·정다미 시모상
1301 0.99532610178 [부고]강승규 모친상 外
552 0.988709926605 [부음] 권덕준 별세 외
1311 0.987241089344 [부고]장삼성 대성 모친상·김규성 남하룡 장모상
1304 0.983856678009 [부고]김상현 주현 동현 부친상
1306 0.983579099178 [부고]김영식 별세
1302 0.983308434486 [부고]강승규 모친상
1303 0.980488359928 [부고]권대혁 경혁 귀현 귀희 정현 부친상·한종우 ·고은아 시부상
1307 0.979106009007 [부고]박건창 건만 건순 모친상
1310 0.978261709213 [부고]이영해 별세
1305 0.968504369259 [부고]김선호 민호 모친상
1308 0.964273750782 [부고]박택상 응상·부친상 고재구 장인상
1461 0.922286987305 [부고]신귀영 별세
1463 0.916531801224 [부고]양승달 별세
1460 0.914805352688 [부고]정운양 운기 운태 운호 정이 모친상·사공숙 장경아 시모상
681 0.90891122818 [부음] 박성연 별세 외
1466 0.899890065193 [부고]김의식 별세·학정 주심 부친상·권진영 시부상
1464 0.895542919636 [부고]정성한 모친상·장지현 시모상
1465

In [28]:
times = pickle.load(open("../datastore/deep_times.p", "rb"))

In [29]:
elapsed_time = 0
for key, value in times.iteritems():
    elapsed_time = elapsed_time + value["elapsed"]
    print key, " ", value["elapsed"]
    
print "\nelapsed_time ", elapsed_time

clustering   4.87321591377
topic   335.42436409
preprocessing   35.5161070824
learning   602.829154015
similarity   97.4006390572

elapsed_time  1076.04348016


# TODO

* 뉴스 유사도 정렬
* 중앙에서 가까운 뉴스 개수 활용하여 이슈 선정